# Breaking Jdaviz
- open 3 to 6 images from JADES - could use also HST images to play with combining gwcs and FITS wcs
- align by WCS
- change orient
- make color image
- create some subsets
- plot nirspec footprint (6 footprints)
- import catalog (need to click search in the UI)
- zoom on target (UI)
- pan and zoom a few times to see where it is with respect to the footprints (UI)
- export some stuff (UI)

In [1]:
from jdaviz import Imviz
from regions import CircleSkyRegion
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import QTable
from astropy.io import fits
import jdaviz
print(jdaviz.__version__)

4.2.2


## All files used here

JADES images and catalog in GOODS-S from v2.0 can be found here: https://archive.stsci.edu/hlsp/jades. They are HLSPs so in principle there is the direct MAST URI.

In [2]:
files = ['mast:HLSP/jades/dr2/goods-s/images/hlsp_jades_jwst_nircam_goods-s-deep_f090w_v2.0_drz.fits',
         'mast:HLSP/jades/dr2/goods-s/images/hlsp_jades_jwst_nircam_goods-s-deep_f115w_v2.0_drz.fits',
         'mast:HLSP/jades/dr2/goods-s/images/hlsp_jades_jwst_nircam_goods-s-deep_f150w_v2.0_drz.fits',
         'mast:HLSP/jades/dr2/goods-s/images/hlsp_jades_jwst_nircam_goods-s-deep_f200w_v2.0_drz.fits',
         'mast:HLSP/jades/dr2/goods-s/images/hlsp_jades_jwst_nircam_goods-s-deep_f277w_v2.0_drz.fits',
         'mast:HLSP/jades/dr2/goods-s/images/hlsp_jades_jwst_nircam_goods-s-deep_f356w_v2.0_drz.fits',
         'mast:HLSP/jades/dr2/goods-s/images/hlsp_jades_jwst_nircam_goods-s-deep_f444w_v2.0_drz.fits'
        ]

## Imviz

In [5]:
imviz = Imviz()
with imviz.batch_load():
    for filepath in files:
        imviz.load_data(filepath, cache=True)
imviz.show()

<IPython.core.display.Javascript object>

Application(config='imviz', docs_link='https://jdaviz.readthedocs.io/en/v4.2.2/imviz/index.html', events=['cal…

## Align

In [6]:
plwcs = imviz.plugins['Orientation']
plwcs.align_by = 'WCS'

In [7]:
plwcs.rotation_angle = 25
#plwcs.new_layer = '25deg' this is throwing an error, but it should be possible given the API hint
plwcs.add_orientation()

## Plot options

In [8]:
plplot = imviz.plugins['Plot Options']
plplot.image_color_mode = 'Color'
plplot.apply_RGB_presets()

In [9]:
datalabels = imviz.data_labels
datalabels

['hlsp_jades_jwst_nircam_goods-s-deep_f090w_v2.0_drz[SCI,1]',
 'hlsp_jades_jwst_nircam_goods-s-deep_f115w_v2.0_drz[SCI,1]',
 'hlsp_jades_jwst_nircam_goods-s-deep_f150w_v2.0_drz[SCI,1]',
 'hlsp_jades_jwst_nircam_goods-s-deep_f200w_v2.0_drz[SCI,1]',
 'hlsp_jades_jwst_nircam_goods-s-deep_f277w_v2.0_drz[SCI,1]',
 'hlsp_jades_jwst_nircam_goods-s-deep_f356w_v2.0_drz[SCI,1]',
 'hlsp_jades_jwst_nircam_goods-s-deep_f444w_v2.0_drz[SCI,1]',
 'Default orientation',
 'CCW 25.00 deg (E-left)']

In [10]:
plplot = imviz.plugins['Plot Options']
for ii in range(0, 7):
    plplot.layer = datalabels[ii]
    plplot.stretch_vmin = 0
    plplot.stretch_vmax = 1.5

## Subsets

In [11]:
reg1_skycoord = SkyCoord(ra=53.16978310613798*u.deg, dec=-27.82401302053590*u.deg)
reg1 = CircleSkyRegion(center=reg1_skycoord, radius=5*u.arcsec)
reg2_skycoord = SkyCoord(ra=53.16315080661268*u.deg, dec=-27.812456485258068*u.deg)
reg2 = CircleSkyRegion(center=reg2_skycoord, radius=5*u.arcsec)
reg3_skycoord = SkyCoord(ra=53.156459566096395*u.deg, dec=-27.810851763504235*u.deg)
reg3 = CircleSkyRegion(center=reg3_skycoord, radius=5*u.arcsec)
reg4_skycoord = SkyCoord(ra=53.16692832905267*u.deg, dec=-27.798776778958917*u.deg)
reg4 = CircleSkyRegion(center=reg4_skycoord, radius=5*u.arcsec)

plsub = imviz.plugins['Subset Tools']
plsub.import_region(reg1, combination_mode='new')
plsub.import_region(reg2, combination_mode='new')
plsub.import_region(reg3, combination_mode='new')
plsub.import_region(reg4, combination_mode='new')


## Footprints

In [12]:
fprint = imviz.plugins['Footprints']
fprint.open_in_tray()
fprint.keep_active = True

pointings = {'name': ['pointing1', 'pointing2', 'pointing3', 'pointing4', 'pointing5', 'pointing6'],
                      'ra': [53.192061669754764, 53.15370709808378, 53.169196860419085, 53.12602624548044, 53.131106556347646, 53.073274945951624],
                      'dec': [-27.771175243855147, -27.75585436112119, -27.817104320074165, -27.802238574939317, -27.869300299918887, -27.85057767095679],
                      'pa': [346.2005, 346.2210, 346.0699, 346.0815, 333.3922, 267.130400]
                     }

num_points = len(pointings['name'])
for i in range(0,num_points):
    fprint.add_overlay(pointings['name'][i])
    fprint.color = 'yellow'
    fprint.preset = 'NIRSpec'
    fprint.ra = pointings['ra'][i]
    fprint.dec = pointings['dec'][i]
    fprint.pa = pointings['pa'][i]

fprint.remove_overlay(lbl='default')

## Catalog
Trying to load the whole thing made my laptop almost crash. I am selecting a subset of sources. Change manually the max number of sources to be more than the length of `subcat`.

In [23]:
catalog = './hlsp_jades_jwst_nircam_goods-s-deep_photometry_v2.0_catalog.fits'

In [25]:
cat = QTable.read(catalog, hdu=3)
cat['sky_centroid'] = SkyCoord(ra=cat['RA']*u.deg, dec=cat['DEC']*u.deg)
subcat = cat[cat['R_KRON']>1.0]
len(subcat)

1264

In [27]:
plcat = imviz.plugins['Catalog Search']
plcat.import_catalog(subcat)